In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score

from sklearn.metrics import confusion_matrix

from sklearn.metrics import classification_report

In [3]:
# Importing the method needed to apply SVM

from sklearn.svm import SVC

## SVM applied to the Default dataset

### A polynomial kernel

__Note from the scikit-learn documentation:__

"Support Vector Machine algorithms are not scale invariant, so it is highly recommended to scale your data."

In [4]:
Default_df= pd.read_csv('C:\\Users\\jheredi2\\Documents\\PythonDataAnalytics\\1-Datasets\\Default.csv')

In [5]:
Default_df_dummies= pd.get_dummies(Default_df,columns=['student'], drop_first=True)

In [6]:
X_train_def, X_test_def, y_train_def, y_test_def= train_test_split (Default_df_dummies.iloc[:,1:], Default_df_dummies['default'], test_size=0.2, random_state=1)

In [7]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [8]:
from sklearn.model_selection import GridSearchCV

In [9]:
pipe_default_CVSearch = make_pipeline(StandardScaler(), SVC())

In [10]:
SVC().get_params().keys()

dict_keys(['C', 'break_ties', 'cache_size', 'class_weight', 'coef0', 'decision_function_shape', 'degree', 'gamma', 'kernel', 'max_iter', 'probability', 'random_state', 'shrinking', 'tol', 'verbose'])

__The hyperparameter C__

Fragment from the ISL book:

"As C increases, the method becomes more tolerant of violations to the margin (the margin is larger) and there are more misclassifications of the training data."

"C=0 amounts to not allowing any misclassification of the trainig data, which is only possible when the classes are separable"

Conversely, as C decreases, the method becomes less tolerant of violations to the margin (the margin narrows) and there are less misclassifications of the training data (but POTENTIALLY MORE OVERFITTING and more misclassifications on the test data !!!)

__The hyperparameter C in scikit learn__: In scikit learn, what they call C is actually 1/(the real C) (confusing, I know !!!)

When you read the interpretation of C in the scikit learn documentation, the relationships are stated differently. For example, they would say:

"_As C increases, there are LESS misclassifications of the training data_" (because in scikit learn, C is actually 1/ the real C)

"The C parameter trades off correct classification of training examples against maximization of the decision function’s margin. For larger values of C, a smaller margin will be accepted if the decision function is better at classifying all training points correctly. A lower C will encourage a larger margin, therefore a simpler decision function, at the cost of training accuracy. In other words C behaves as a regularization parameter in the SVM."

From the scikit-learn documentation:

"C is 1 by default and it’s a reasonable default choice. If you have a lot of noisy observations you should decrease it: decreasing C corresponds to more regularization."

__Note__: When fitting a polynomial kernel, we could also tune another hyperparameter called gamma. We won't tune for two reasons:

a) It is not fundamental to do so with a polynomial: you could try different values of gamma and see what happens, but it not fundamental that you do so. We can always use the default value of gamma.

b) Tuning both C and gamma is VERY computationally intense... So need to do tune them both if it is not fundamental

Tuning gamma when a radial basis kernel is used is more fundamental (more advisable to do) than doing it with a poynomial kernel.

In [11]:
# Real C (C from the book) (I limited it to only a few values because the computation is very intense)

real_c= np.array ([1/100, 1/2, 1, 5, 10, 100])

In [12]:
real_c

array([1.e-02, 5.e-01, 1.e+00, 5.e+00, 1.e+01, 1.e+02])

In [13]:
c_hyperparameter= 1/real_c

In [14]:
c_hyperparameter

array([1.e+02, 2.e+00, 1.e+00, 2.e-01, 1.e-01, 1.e-02])

In [15]:
hyperparam_grid_poly = {
    'svc__C': c_hyperparameter,
    'svc__class_weight': ['balanced'],
    'svc__degree':[1,2,3,4], # degree 1 correspond to the linear kernel (= support vector classifier)
    'svc__kernel':['poly']
}

In [16]:
grid_search_default= GridSearchCV(estimator= pipe_default_CVSearch, param_grid=hyperparam_grid_poly, cv=5, scoring='accuracy')

In [17]:
grid_search_default.fit(X_train_def, y_train_def)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('svc', SVC())]),
             param_grid={'svc__C': array([1.e+02, 2.e+00, 1.e+00, 2.e-01, 1.e-01, 1.e-02]),
                         'svc__class_weight': ['balanced'],
                         'svc__degree': [1, 2, 3, 4], 'svc__kernel': ['poly']},
             scoring='accuracy')

In [18]:
grid_search_default.best_params_

{'svc__C': 0.01,
 'svc__class_weight': 'balanced',
 'svc__degree': 3,
 'svc__kernel': 'poly'}

In [19]:
pipe_default_poly_svc = make_pipeline(StandardScaler(), SVC(kernel='poly', degree=3, C=0.01, class_weight='balanced'))

In [20]:
pipe_default_poly_svc.fit (X_train_def, y_train_def)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=0.01, class_weight='balanced', kernel='poly'))])

In [21]:
y_predicted_test_default1= pipe_default_poly_svc.predict(X_test_def)

In [22]:
confusion_matrix (y_test_def, y_predicted_test_default1)

array([[1781,  160],
       [  15,   44]], dtype=int64)

In [23]:
print (classification_report (y_test_def, y_predicted_test_default1))

              precision    recall  f1-score   support

          No       0.99      0.92      0.95      1941
         Yes       0.22      0.75      0.33        59

    accuracy                           0.91      2000
   macro avg       0.60      0.83      0.64      2000
weighted avg       0.97      0.91      0.93      2000



__Choosing the hyperparameters based on f1_score__

In [24]:
from sklearn.metrics import f1_score

from sklearn.metrics import make_scorer

In [25]:
f1_scorer = make_scorer(f1_score, pos_label='Yes')

In [27]:
grid_search_default_f1score= GridSearchCV(estimator= pipe_default_CVSearch, param_grid= hyperparam_grid_poly, cv=5, scoring=f1_scorer)

In [ ]:
grid_search_default_f1score.fit(X_train_def, y_train_def)

In [ ]:
grid_search_default_f1score.best_params_

Same hyperparameter values we got when we based the search on accuracy. Therefore, no need to continue.

### A Radial Basis Function (RBF) kernel

__Note from the scikit-learn documentation:__

" When training an SVM with the Radial Basis Function (RBF) kernel, two parameters must be considered: __C and gamma__." 

"The parameter C, common to all SVM kernels, trades off misclassification of training examples against performance on test data."

"Gamma defines how much influence a single training example has. The larger gamma is, the closer other examples must be to be affected."

"Intuitively, the gamma parameter defines how far the influence of a single training example reaches, with low values meaning ‘far’ and high values meaning ‘close’"

"Proper choice of C and gamma is critical to the SVM’s performance. One is advised to use GridSearchCV to find C and gamma."

What values of gamma to try?

"In practice, a logarithmic (base 10) grid from 10^-3 to 10^ 3 is enough"

https://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html

In [4]:
gamma_range = np.logspace(-3, 3, 5)
gamma_range

array([1.00000000e-03, 3.16227766e-02, 1.00000000e+00, 3.16227766e+01,
       1.00000000e+03])

What does np.logspace() give us?

https://numpy.org/doc/stable/reference/generated/numpy.logspace.html

In [5]:
for i in gamma_range:
    print (np.log10(i))

-3.0
-1.5
0.0
1.5
3.0


In [29]:
hyperparam_grid_default_rbf = {
    'svc__C': c_hyperparameter,
    'svc__class_weight': ['balanced'],
    'svc__kernel':['rbf'],
    'svc__gamma': gamma_range
}

In [30]:
grid_search_default_rbf= GridSearchCV(estimator= pipe_default_CVSearch, param_grid=hyperparam_grid_default_rbf, cv=5, scoring='accuracy')

In [31]:
grid_search_default_rbf.fit(X_train_def, y_train_def)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('svc', SVC())]),
             param_grid={'svc__C': array([1.e+02, 2.e+00, 1.e+00, 2.e-01, 1.e-01, 1.e-02]),
                         'svc__class_weight': ['balanced'],
                         'svc__gamma': array([1.00000000e-03, 3.16227766e-02, 1.00000000e+00, 3.16227766e+01,
       1.00000000e+03]),
                         'svc__kernel': ['rbf']},
             scoring='accuracy')

In [32]:
grid_search_default_rbf.best_params_

{'svc__C': 2.0,
 'svc__class_weight': 'balanced',
 'svc__gamma': 1000.0,
 'svc__kernel': 'rbf'}

In [33]:
pipe_default_rbf_svc = make_pipeline(StandardScaler(), SVC(kernel='rbf', C= 2, gamma= 1000, class_weight='balanced'))

In [34]:
pipe_default_rbf_svc.fit(X_train_def, y_train_def)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=2, class_weight='balanced', gamma=1000))])

In [35]:
y_predicted_test_default2= pipe_default_rbf_svc.predict(X_test_def)

In [36]:
confusion_matrix (y_test_def, y_predicted_test_default2)

array([[1925,   16],
       [  55,    4]], dtype=int64)

In [37]:
print (classification_report(y_test_def, y_predicted_test_default2))

              precision    recall  f1-score   support

          No       0.97      0.99      0.98      1941
         Yes       0.20      0.07      0.10        59

    accuracy                           0.96      2000
   macro avg       0.59      0.53      0.54      2000
weighted avg       0.95      0.96      0.96      2000



__Choosing the hyperparameters based on f1_score__ (DO IT AT HOME)

## SVM applied to the Sonar dataset

### A polynomial kernel

In [38]:
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/sonar.csv"

In [39]:
sonar_df = pd.read_csv(url, header=None)

In [40]:
sonar_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 61 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       208 non-null    float64
 1   1       208 non-null    float64
 2   2       208 non-null    float64
 3   3       208 non-null    float64
 4   4       208 non-null    float64
 5   5       208 non-null    float64
 6   6       208 non-null    float64
 7   7       208 non-null    float64
 8   8       208 non-null    float64
 9   9       208 non-null    float64
 10  10      208 non-null    float64
 11  11      208 non-null    float64
 12  12      208 non-null    float64
 13  13      208 non-null    float64
 14  14      208 non-null    float64
 15  15      208 non-null    float64
 16  16      208 non-null    float64
 17  17      208 non-null    float64
 18  18      208 non-null    float64
 19  19      208 non-null    float64
 20  20      208 non-null    float64
 21  21      208 non-null    float64
 22  22

In [41]:
X_train_sonar, X_test_sonar, y_train_sonar, y_test_sonar= train_test_split (sonar_df.iloc[:,:-1], sonar_df.iloc[:,-1], test_size=0.2, random_state=1)

In [42]:
pipe_sonar_CVSearch = make_pipeline(StandardScaler(), SVC())

In [43]:
real_c_sonar= np.array ([1/100, 1/50, 1/10, 1/2, 1, 5, 10, 20, 50, 100, 200, 500, 1000])

In [44]:
 c_hyperparameter_sonar= 1/ real_c_sonar

In [45]:
hyperparam_grid_poly_sonar = {
    'svc__C': c_hyperparameter_sonar,
    'svc__class_weight': ['balanced'],
    'svc__degree':[1,2,3,4,5], # degree 1 correspond to the linear kernel
    'svc__kernel':['poly']
}

In [46]:
grid_search_sonar= GridSearchCV(estimator= pipe_sonar_CVSearch, param_grid=hyperparam_grid_poly_sonar, cv= 5, scoring='accuracy')

In [47]:
grid_search_sonar.fit(X_train_sonar, y_train_sonar)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('svc', SVC())]),
             param_grid={'svc__C': array([1.e+02, 5.e+01, 1.e+01, 2.e+00, 1.e+00, 2.e-01, 1.e-01, 5.e-02,
       2.e-02, 1.e-02, 5.e-03, 2.e-03, 1.e-03]),
                         'svc__class_weight': ['balanced'],
                         'svc__degree': [1, 2, 3, 4, 5],
                         'svc__kernel': ['poly']},
             scoring='accuracy')

In [48]:
grid_search_sonar.best_params_

{'svc__C': 1.0,
 'svc__class_weight': 'balanced',
 'svc__degree': 3,
 'svc__kernel': 'poly'}

In [49]:
pipe_sonar_poly_svc = make_pipeline(StandardScaler(), SVC(kernel='poly', degree=3, C= 1, class_weight='balanced'))

In [50]:
pipe_sonar_poly_svc.fit(X_train_sonar, y_train_sonar)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=1, class_weight='balanced', kernel='poly'))])

In [51]:
y_predicted_test_sonar1= pipe_sonar_poly_svc.predict(X_test_sonar)

In [52]:
confusion_matrix (y_test_sonar, y_predicted_test_sonar1)

array([[16,  4],
       [ 3, 19]], dtype=int64)

In [53]:
print (classification_report (y_test_sonar, y_predicted_test_sonar1))

              precision    recall  f1-score   support

           M       0.84      0.80      0.82        20
           R       0.83      0.86      0.84        22

    accuracy                           0.83        42
   macro avg       0.83      0.83      0.83        42
weighted avg       0.83      0.83      0.83        42



### A Radial Basis Function (RBF) kernel

In [54]:
gamma_range_sonar = np.logspace(-3, 3, 20)
gamma_range_sonar

array([1.00000000e-03, 2.06913808e-03, 4.28133240e-03, 8.85866790e-03,
       1.83298071e-02, 3.79269019e-02, 7.84759970e-02, 1.62377674e-01,
       3.35981829e-01, 6.95192796e-01, 1.43844989e+00, 2.97635144e+00,
       6.15848211e+00, 1.27427499e+01, 2.63665090e+01, 5.45559478e+01,
       1.12883789e+02, 2.33572147e+02, 4.83293024e+02, 1.00000000e+03])

In [55]:
hyperparam_grid_sonar_rbf = {
    'svc__C': c_hyperparameter_sonar,
    'svc__class_weight': ['balanced'],
    'svc__kernel':['rbf'],
    'svc__gamma': gamma_range
}

In [56]:
grid_search_sonar_rbf= GridSearchCV(estimator= pipe_sonar_CVSearch, param_grid= hyperparam_grid_sonar_rbf, cv=5, scoring='accuracy')

In [57]:
grid_search_sonar_rbf.fit(X_train_sonar, y_train_sonar)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('svc', SVC())]),
             param_grid={'svc__C': array([1.e+02, 5.e+01, 1.e+01, 2.e+00, 1.e+00, 2.e-01, 1.e-01, 5.e-02,
       2.e-02, 1.e-02, 5.e-03, 2.e-03, 1.e-03]),
                         'svc__class_weight': ['balanced'],
                         'svc__gamma': array([1.00000000e-03, 3.16227766e-02, 1.00000000e+00, 3.16227766e+01,
       1.00000000e+03]),
                         'svc__kernel': ['rbf']},
             scoring='accuracy')

In [58]:
grid_search_sonar_rbf.best_params_

{'svc__C': 100.0,
 'svc__class_weight': 'balanced',
 'svc__gamma': 0.03162277660168379,
 'svc__kernel': 'rbf'}

In [59]:
pipe_sonar_rbf_svc = make_pipeline(StandardScaler(), SVC(kernel='rbf', C= 100, gamma= 0.03162277660168379, class_weight='balanced'))

In [60]:
pipe_sonar_rbf_svc.fit(X_train_sonar, y_train_sonar)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc',
                 SVC(C=100, class_weight='balanced',
                     gamma=0.03162277660168379))])

In [61]:
y_predicted_test_sonar2= pipe_sonar_rbf_svc.predict(X_test_sonar)

In [62]:
print (classification_report (y_test_sonar, y_predicted_test_sonar2))

              precision    recall  f1-score   support

           M       0.77      1.00      0.87        20
           R       1.00      0.73      0.84        22

    accuracy                           0.86        42
   macro avg       0.88      0.86      0.86        42
weighted avg       0.89      0.86      0.86        42



## SVM applied to the Tips dataset

__WORK ON THIS INDIVIDUALLY NOW IN CLASS !!!__

EVERYONE MUST WORK ON THIS INDIVIDUALLY !!! 

CODE WILL BE DISCUSSED AND GUIDANCE PROVIDED, BUT CODE WILL NOT BE SHARED !!!

__First)__ Use polynomial kernel and select based on f1-score

__Second)__ Use radial basis functions kernel and select based on f1-score

__Third)__ Use radial basis functions kernel and select based on overall accuracy

In [ ]:
Tips_df= pd.read_csv('C:\\Users\\jheredi2\\Documents\\PythonDataAnalytics\\1-Datasets\\tips.csv')

In [ ]:
Tips_df_dummies= pd.get_dummies(Tips_df,columns=['sex','smoker','day','time'], drop_first=True)

In [ ]:
Tips_df_dummies['great_tip']=((Tips_df_dummies['tip']/Tips_df_dummies['total_bill'])>=0.20)*1

In [ ]:
Tips_df_dummies.info()

In [ ]:
X_train_tip, X_test_tip, y_train_tip, y_test_tip= train_test_split (Tips_df_dummies.iloc[:,np.r_[0, 2:9]], Tips_df_dummies.iloc[:,-1], test_size=0.2, random_state=1)

POLYNOMIAL

RADIAL BASIS FUNCTION

RADIAL BASIS FUNCTION based on accuracy